# Boosting

In [1]:
library(tidyverse)

#install.packages('hablar')
#library(hablar)

#install.packages("tidyr")
#install.packages("rlang")
library(tidyr)

#install.packages('scales')
library(scales)

library(LICORS)

#install.packages("slider")
library(slider)

library(glmnet)

library(gbm)

Registered S3 method overwritten by 'rvest':
  method            from
  read_xml.response xml2
── Attaching packages ───────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.4     ✔ dplyr   1.0.7
✔ tidyr   1.1.3     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.4.0
── Conflicts ──────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Attaching package: ‘scales’

The following object is masked from ‘package:purrr’:

    discard

The following object is masked from ‘package:readr’:

    col_factor

Loading required package: Matrix

Attaching package: ‘Matrix’

The following objects are masked from ‘package:tidyr’:

    expand, pack, unpack

Loading required package: foreach

Attaching package: ‘foreach’

The following objects are masked from ‘package:purrr’:

    accumulate, when

Loaded glmnet 2.0-16

Loaded gbm 2.1.8


In [2]:
sum_control_ATAC_bin_rm = read.csv("data/sum_control_ATAC_bin_rm.csv")

In [3]:
head(sum_control_ATAC_bin_rm)

X30,X31,X32,X33,X34,X35,X36,X37,X38,X39,...,X692,X693,X694,X695,X696,X697,X698,X699,X700,ATAC_val
8.699646e-07,3.479859e-06,2.609894e-06,0.000000e+00,8.699646e-07,8.699646e-07,1.739929e-06,1.739929e-06,0.000000e+00,2.609894e-06,...,0,0,0,0,0,0,0,0,0.000000e+00,0.5931657
8.313727e-07,8.313727e-07,0.000000e+00,8.313727e-07,1.662745e-06,8.313727e-07,8.313727e-07,5.819609e-06,3.325491e-06,3.325491e-06,...,0,0,0,0,0,0,0,0,0.000000e+00,0.4089315
8.705774e-07,1.741155e-06,8.705774e-07,8.705774e-07,0.000000e+00,0.000000e+00,2.611732e-06,3.482309e-06,0.000000e+00,8.705774e-07,...,0,0,0,0,0,0,0,0,8.705774e-07,0.3058228
0.000000e+00,1.700685e-06,0.000000e+00,8.503423e-07,8.503423e-07,1.700685e-06,8.503423e-07,0.000000e+00,8.503423e-07,8.503423e-07,...,0,0,0,0,0,0,0,0,0.000000e+00,0.4764137
0.000000e+00,9.416577e-07,0.000000e+00,0.000000e+00,9.416577e-07,0.000000e+00,9.416577e-07,0.000000e+00,2.824973e-06,0.000000e+00,...,0,0,0,0,0,0,0,0,0.000000e+00,0.2862775
0.000000e+00,0.000000e+00,0.000000e+00,4.221123e-06,1.688449e-06,1.688449e-06,8.442245e-07,0.000000e+00,2.532674e-06,0.000000e+00,...,0,0,0,0,0,0,0,0,0.000000e+00,0.3169988


In [4]:
set.seed(0)

fit <- gbm(formula  = ATAC_val ~ ., 
               data = sum_control_ATAC_bin_rm, 
            n.trees =500, 
           cv.folds = 10)

Distribution not specified, assuming gaussian ...


ERROR: Error in serialize(data, node$con): error writing to connection


In [ ]:
fit

## Plot the training error and the CV RMSE

In [ ]:
pd <- tibble(rmse_cv = sqrt(fit$cv.error), 
             rmse_train = sqrt(fit$train.error)) %>%
  mutate(tree = row_number()) %>%
  pivot_longer(names_to = "key", values_to = "value", -tree)

ggplot(pd, aes(x=tree, y=value, color=key)) + 
  geom_line() + 
  geom_point() + 
  NULL

In [ ]:
gbm.perf(fit, method = "cv")

In [ ]:
fit$train.error[which.min(sqrt(fit$train.error))]

fit$cv.error[which.min(sqrt(fit$cv.error))]


In [ ]:
predictions <- fit$cv.fitted
observed <- as.data.frame(sum_control_ATAC_bin_rm$ATAC_val)

In [ ]:
combined <- cbind(predictions, observed)
colnames(combined) <- c("predictions", "observed")
head(combined)

In [ ]:
ggplot(data = combined, aes(x = observed, y = predictions)) + 
    geom_point(size = 0.5) + 
    geom_smooth(method = "lm", formula = y~x)

In [ ]:
cor(combined$observed, combined$predictions)

# Extract the predictors

In [ ]:
#summary(fit)